<h1>Cloning GitHub Repo</h1>

In [1]:
#----------Cloning GitHub Repo------------
!git clone https://github.com/SimoneRoma21/PKMN_VictoryBinaryClassification.git
#!git pull

Cloning into 'PKMN_VictoryBinaryClassification'...
remote: Enumerating objects: 628, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 628 (delta 100), reused 233 (delta 85), pack-reused 370 (from 1)
Receiving objects: 100% (628/628), 32.39 MiB | 24.92 MiB/s, done.
Resolving deltas: 100% (353/353), done.


In [2]:
#---------Appending the right path----------
import sys

# Make sure to replace the repository folder with the actual folder name
repo_path = ['/kaggle/working/PKMN_VictoryBinaryClassification/py','/kaggle/working/PKMN_VictoryBinaryClassification/data']
if repo_path not in sys.path:
    sys.path.append(repo_path)
print(sys.path)

%cd /kaggle/working/PKMN_VictoryBinaryClassification/py

['/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/kaggle/input/fds-pokemon-battles-prediction-2025', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', ['/kaggle/working/PKMN_VictoryBinaryClassification/py', '/kaggle/working/PKMN_VictoryBinaryClassification/data']]
/kaggle/working/PKMN_VictoryBinaryClassification/py


<H1>Imports</H1>

In [4]:
import json
import pandas as pd
from dataset.dataset_construction import Feature, FeaturePipeline
from dataset.csv_utilities import *
from dataset.extract_utilities import *
from ModelTrainer import ModelTrainer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier



<H1>Feature Selected</H1>

In [5]:
selected_features = [
        
        #------Stats Features---------#
        Feature.P1_FINAL_TEAM_HP, 
        Feature.P2_FINAL_TEAM_HP, 
        Feature.MEAN_SPE_LAST, 
        Feature.MEAN_HP_LAST, 
        Feature.MEAN_ATK_LAST, 
        Feature.MEAN_SPA_LAST, 
        Feature.MEAN_STATS_LAST, 
        Feature.MEAN_CRIT,

        #------Ratio on Stats Features--------#
        Feature.HP_BULK_RATIO,
        Feature.SPE_ATK_RATIO,
        Feature.OFF_DEF_RATIO,
        Feature.OFF_SPAD_RATIO,

        #-------Differential Features on Stats---#
        Feature.HP_TREND_DIFF,
        Feature.ATK_TREND_DIFF,
        Feature.SPA_TREND_DIFF,
        Feature.SPE_TREND_DIFF,
    
        #---Feature Infos During Battle----#
        Feature.P1_ALIVE_PKMN, 
        Feature.P2_ALIVE_PKMN, 
        Feature.P1_SWITCHES_COUNT, 
        Feature.P2_SWITCHES_COUNT,
    
        Feature.P1_AVG_HP_WHEN_SWITCHING, 
        Feature.P2_AVG_HP_WHEN_SWITCHING, 
        Feature.P1_MAX_DEBUFF_RECEIVED,
        Feature.P2_MAX_DEBUFF_RECEIVED,
        Feature.P1_AVG_MOVE_POWER, 
        Feature.P2_AVG_MOVE_POWER, 
        Feature.AVG_MOVE_POWER_DIFFERENCE, 
        Feature.P1_OFFENSIVE_RATIO, 
        Feature.P2_OFFENSIVE_RATIO, 
        Feature.OFFENSIVE_RATIO_DIFFERENCE, 
        Feature.P1_MOVED_FIRST_COUNT, 
        Feature.P2_MOVED_FIRST_COUNT, 
        Feature.SPEED_ADVANTAGE_RATIO, 

        #----Feature Status of Pokemons----#
        Feature.P1_FROZEN_PKMN, 
        Feature.P2_FROZEN_PKMN, 
        Feature.P1_PARALIZED_PKMN, 
        Feature.P2_PARALIZED_PKMN, 
        Feature.P1_SLEEP_PKMN, 
        Feature.P2_SLEEP_PKMN, 
        Feature.P1_POISON_PKMN, 
        Feature.P2_POISON_PKMN,  
        Feature.P1_BURNED_PKMN, 
        Feature.P2_BURNED_PKMN, 
        
        #----Feature Pokemon Moves----#
        Feature.P1_PKMN_REFLECT, 
        Feature.P2_PKMN_REFLECT, 
        Feature.P1_PKMN_REST, 
        Feature.P2_PKMN_REST, 
        Feature.P1_PKMN_EXPLOSION, 
        Feature.P2_PKMN_EXPLOSION, 
        Feature.P1_PKMN_THUNDERWAVE, 
        Feature.P2_PKMN_THUNDERWAVE, 
        Feature.P1_PKMN_RECOVER, 
        Feature.P2_PKMN_RECOVER, 
        Feature.P1_PKMN_TOXIC, 
        Feature.P2_PKMN_TOXIC, 
        Feature.P1_PKMN_FIRESPIN, 
        Feature.P2_PKMN_FIRESPIN,          
    ]
    

<h1>Main code for training and evaluation</h1>


### First Subsmission

This submission uses GridSearch on a LogisticRegression with 0.2/0.8 ratio for test/train. CrossValidation uses 5 fold and is applied by the GridSearch itself. On the public the submission is called first_submission.csv. Locally we have 84.89 training accuracy and 85.55 evaluation accuracy. On the public we get 84.93.

In [ ]:
def evaluate_test_set(trainer: ModelTrainer, feature_list: list, test_file_path: str):

    feature_pipeline = FeaturePipeline(feature_list, cache_dir="../data/test_feature_cache")

    print("\nLoading test data...")
    test_data = []
    with open(test_file_path, 'r') as f:
        for line in f:
            test_data.append(json.loads(line))

    # Extract features from test set
    print("\nExtracting features from test data...")
    test_df = feature_pipeline.extract_features(test_data, show_progress=True)

    X_test = test_df.drop(['battle_id'], axis=1, errors='ignore')

    # Predict on test set
    predictions = trainer.predict(X_test)

    submission = pd.DataFrame({
        'battle_id': test_df['battle_id'],
        'player_won': predictions
    })
    submission.to_csv('/kaggle/working/first_submission.csv', index=False)


feature_pipeline = FeaturePipeline(selected_features)

train_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/train.jsonl'
test_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/test.jsonl'
train_out_path="predict_csv/train_features_extracted.csv"

print("Loading training data...")
train_data = []
with open(train_file_path, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Extract the features for train_set
print("\nExtracting features from training data...")
train_df = feature_pipeline.extract_features(train_data)
print("\nTraining features preview:")
print(train_df.head())

# Save dataset in a CSV file
train_df.to_csv(train_out_path, index=False)

#---------------Model Training and Evaluation Code------------------------

# Remove row 4877 from the train dataset
train_df = train_df.drop(index=4877)
X_train = train_df.drop(['battle_id', 'player_won'], axis=1)
y_train = train_df['player_won']


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Pipeline with scaler and model
print("\nCreating pipeline with RobustScaler and LogisticRegression...")
pipeline = Pipeline([
    ('scaler',RobustScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=2000)),
])

#Grid Search for Logistic Regression 
param_grid={
     'classifier__C':[0.01,0.1,1,10,100],
     'classifier__penalty': ['l1','l2'],
     'classifier__solver':['liblinear','saga'],
     'classifier__max_iter':[1000,2000]
}

grid_logreg = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='roc_auc',
    # scoring='accuracy',
    n_jobs=-1,  
    cv=5,            # 5-fold cross-validation, more on this later
    refit=True,      # retrain the best model on the full training set
    return_train_score=True
)


trainer = ModelTrainer(grid_logreg)
trainer.train(X_tr, y_tr)
trainer.evaluate(X_val, y_val)

print("Best CV score:", grid_logreg.best_score_)
print("Best params:", grid_logreg.best_params_)


# ------------------ Evaluate on Test Set -----------------
evaluate_test_set(trainer, selected_features, test_file_path)

### Second Subsmission

This submission uses GridSearch on a LogisticRegression with 0.1/0.9 ratio for test/train. CrossValidation uses 5 fold and is applied by the GridSearch itself. On the public the submission is called second_submission.csv. Locally we have 85.03 training accuracy and 85.30 evaluation accuracy. On the public we get 84.73.


In [ ]:
def evaluate_test_set(trainer: ModelTrainer, feature_list: list, test_file_path: str):

    feature_pipeline = FeaturePipeline(feature_list, cache_dir="../data/test_feature_cache")

    print("\nLoading test data...")
    test_data = []
    with open(test_file_path, 'r') as f:
        for line in f:
            test_data.append(json.loads(line))

    # Extract features from test set
    print("\nExtracting features from test data...")
    test_df = feature_pipeline.extract_features(test_data, show_progress=True)

    X_test = test_df.drop(['battle_id'], axis=1, errors='ignore')

    # Predict on test set
    predictions = trainer.predict(X_test)

    submission = pd.DataFrame({
        'battle_id': test_df['battle_id'],
        'player_won': predictions
    })
    submission.to_csv('/kaggle/working/second_submission.csv', index=False)



feature_pipeline = FeaturePipeline(selected_features)

train_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/train.jsonl'
test_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/test.jsonl'
train_out_path="predict_csv/train_features_extracted.csv"

print("Loading training data...")
train_data = []
with open(train_file_path, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Extract the features for train_set
print("\nExtracting features from training data...")
train_df = feature_pipeline.extract_features(train_data)
print("\nTraining features preview:")
print(train_df.head())

# Save dataset in a CSV file
train_df.to_csv(train_out_path, index=False)

#---------------Model Training and Evaluation Code------------------------

# Remove row 4877 from the train dataset
train_df = train_df.drop(index=4877)
X_train = train_df.drop(['battle_id', 'player_won'], axis=1)
y_train = train_df['player_won']


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Pipeline with scaler and model
print("\nCreating pipeline with RobustScaler and LogisticRegression...")
pipeline = Pipeline([
    ('scaler',RobustScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=2000)),
])

#Grid Search for Logistic Regression 
param_grid={
     'classifier__C':[0.01,0.1,1,10,100],
     'classifier__penalty': ['l1','l2'],
     'classifier__solver':['liblinear','saga'],
     'classifier__max_iter':[1000,2000]
}

grid_logreg = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='roc_auc',
    # scoring='accuracy',
    n_jobs=-1,  
    cv=5,            # 5-fold cross-validation, more on this later
    refit=True,      # retrain the best model on the full training set
    return_train_score=True
)


trainer = ModelTrainer(grid_logreg)
trainer.train(X_tr, y_tr)
trainer.evaluate(X_val, y_val)

print("Best CV score:", grid_logreg.best_score_)
print("Best params:", grid_logreg.best_params_)


# ------------------ Evaluate on Test Set -----------------

evaluate_test_set(trainer, selected_features, test_file_path)

### Third Subsmission
This submission uses ModelStacking where the base estimators are LogisticRegression, SGDClassifier, XGBClassifier and RandomForest. We used 2 linear models and 2 non-linear models to combine the advantages of both types. Each of the base estimators has been fine-tuned separately using GridSearchCV on the same training data before being included in the stacking ensemble. The meta model is LogisticRegression. Split ratio is 0.2/0.8 for test/train. CrossValidation uses 5 fold and is applied by the GridSearch itself. On the public the submission is called third_submission.csv. Locally we have 85.15 training accuracy and 85.45 evaluation accuracy. On the public we get 84.46.

In [6]:
def evaluate_test_set(trainer: ModelTrainer, feature_list: list, test_file_path: str):

    feature_pipeline = FeaturePipeline(feature_list, cache_dir="../data/test_feature_cache")

    print("\nLoading test data...")
    test_data = []
    with open(test_file_path, 'r') as f:
        for line in f:
            test_data.append(json.loads(line))

    # Extract features from test set
    print("\nExtracting features from test data...")
    test_df = feature_pipeline.extract_features(test_data, show_progress=True)

    X_test = test_df.drop(['battle_id'], axis=1, errors='ignore')

    # Predict on test set
    predictions = trainer.predict(X_test)

    submission = pd.DataFrame({
        'battle_id': test_df['battle_id'],
        'player_won': predictions
    })
    submission.to_csv('/kaggle/working/third_submission.csv', index=False)

feature_pipeline = FeaturePipeline(selected_features)

train_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/train.jsonl'
test_file_path = '/kaggle/input/fds-pokemon-battles-prediction-2025/test.jsonl'
train_out_path="predict_csv/train_features_extracted.csv"

print("Loading training data...")
train_data = []
with open(train_file_path, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Extract the features for train_set
print("\nExtracting features from training data...")
train_df = feature_pipeline.extract_features(train_data)
print("\nTraining features preview:")
print(train_df.head())

# Save dataset in a CSV file
train_df.to_csv(train_out_path, index=False)

#---------------Model Training and Evaluation Code------------------------

# Remove row 4877 from the train dataset
train_df = train_df.drop(index=4877)
X_train = train_df.drop(['battle_id', 'player_won'], axis=1)
y_train = train_df['player_won']


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Pipelines: Scaling only the linear models
pipe_lr = Pipeline([
    ('scaler', RobustScaler()),
    ('lr', LogisticRegression(max_iter=1000,C=1,penalty='l1',solver='liblinear',random_state=42))
])

pipe_sgd = Pipeline([
    ('scaler', RobustScaler()),
    ('sgd', SGDClassifier(
        loss='modified_huber',
        penalty='l1',
        alpha=0.001,
        learning_rate='adaptive',
        eta0=0.01,
        max_iter=1000,
        tol=0.0001,
        random_state=42
    ))
])

# Tree and Boosting
xgb = XGBClassifier(
    eval_metric='logloss',
    random_state=42,
    colsample_bytree=0.8,
    gamma=0.3,
    learning_rate=0.05,
    max_depth=3,
    min_child_weight=5,
    n_estimators=400,
    reg_alpha=0,
    reg_lambda=2,
    subsample=0.8
    )

rf = RandomForestClassifier(
    random_state=42, bootstrap=True,
    max_depth=12, max_features='sqrt',
    min_samples_leaf=3, min_samples_split=5, n_estimators=300
    )

# Base estimators
base_estimators = [
    ('lr', pipe_lr),
    ('xgb', xgb),
    ('rf', rf),
    ('sgd', pipe_sgd)
]

# Meta model with gridSearchCV for the hyperparameters
meta = LogisticRegression(random_state=42)

stack = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta,
    cv=5,
    stack_method='predict_proba',   
    n_jobs=-1,
    passthrough=False
)
param_grid = {
'final_estimator__C': [0.01, 0.1, 1, 10, 100],
'final_estimator__penalty': ['l1', 'l2'],
'final_estimator__solver': ['liblinear', 'saga'],
'final_estimator__max_iter': [1000, 2000]
}

gs = GridSearchCV(stack, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

trainer = ModelTrainer(gs)
trainer.train(X_tr,y_tr)
trainer.evaluate(X_val,y_val)

print("Best CV score:", gs.best_score_)
print("Best params:", gs.best_params_)

# ------------------ Evaluate on Test Set -----------------

evaluate_test_set(trainer, selected_features, test_file_path)

Loading training data...

Extracting features from training data...


Extracting features:  18%|█▊        | 10/57 [00:00<00:00, 95.64it/s]

Loading p1_final_team_hp from cache...
Loading p2_final_team_hp from cache...
Loading mean_spe_last from cache...
Loading mean_hp_last from cache...
Loading mean_atk_last from cache...
Loading mean_spa_last from cache...
Loading mean_stats_last from cache...
Loading mean_crit from cache...
Loading hp_bulk_ratio from cache...
Loading spe_atk_ratio from cache...
Loading off_def_ratio from cache...
Loading off_spad_ratio from cache...
Loading hp_trend_diff from cache...
Loading atk_trend_diff from cache...
Loading spa_trend_diff from cache...
Loading spe_trend_diff from cache...
Loading p1_alive_pkmn from cache...
Loading p2_alive_pkmn from cache...
Loading p1_switches_count from cache...
Loading p2_switches_count from cache...
Loading p1_avg_hp_when_switching from cache...
Loading p2_avg_hp_when_switching from cache...
Loading p1_max_debuff_received from cache...
Loading p2_max_debuff_received from cache...


Extracting features: 100%|██████████| 57/57 [00:00<00:00, 178.88it/s]

Loading p1_avg_move_power from cache...
Loading p2_avg_move_power from cache...
Loading avg_move_power_difference from cache...
Loading p1_offensive_ratio from cache...
Loading p2_offensive_ratio from cache...
Loading offensive_ratio_difference from cache...
Loading p1_moved_first_count from cache...
Loading p2_moved_first_count from cache...
Loading speed_advantage_ratio from cache...
Loading p1_frozen_pkmn from cache...
Loading p2_frozen_pkmn from cache...
Loading p1_paralized_pkmn from cache...
Loading p2_paralized_pkmn from cache...
Loading p1_sleep_pkmn from cache...
Loading p2_sleep_pkmn from cache...
Loading p1_poison_pkmn from cache...
Loading p2_poison_pkmn from cache...
Loading p1_burned_pkmn from cache...
Loading p2_burned_pkmn from cache...
Loading p1_pkmn_reflect from cache...
Loading p2_pkmn_reflect from cache...
Loading p1_pkmn_rest from cache...
Loading p2_pkmn_rest from cache...
Loading p1_pkmn_explosion from cache...
Loading p2_pkmn_explosion from cache...
Loading p1_

Training model...


/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


Training Accuracy: 0.8514
✓ Training complete!

MODEL EVALUATION

Accuracy: 0.8545

ROC-AUC Score:, 0.8545

Confusion Matrix:
[[844 142]
 [149 865]]

True Negatives:   844  |  False Positives:  142
False Negatives:  149  |  True Positives:   865

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       986
           1       0.86      0.85      0.86      1014

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000

Best CV score: 0.917767175505519
Best params: {'final_estimator__C': 0.01, 'final_estimator__max_iter': 1000, 'final_estimator__penalty': 'l1', 'final_estimator__solver': 'saga'}

Loading test data...

Extracting features from test data...


Extracting features: 100%|██████████| 57/57 [00:00<00:00, 344.38it/s]

Loading p1_final_team_hp from cache...
Loading p2_final_team_hp from cache...
Loading mean_spe_last from cache...
Loading mean_hp_last from cache...
Loading mean_atk_last from cache...
Loading mean_spa_last from cache...
Loading mean_stats_last from cache...
Loading mean_crit from cache...
Loading hp_bulk_ratio from cache...
Loading spe_atk_ratio from cache...
Loading off_def_ratio from cache...
Loading off_spad_ratio from cache...
Loading hp_trend_diff from cache...
Loading atk_trend_diff from cache...
Loading spa_trend_diff from cache...
Loading spe_trend_diff from cache...
Loading p1_alive_pkmn from cache...
Loading p2_alive_pkmn from cache...
Loading p1_switches_count from cache...
Loading p2_switches_count from cache...
Loading p1_avg_hp_when_switching from cache...
Loading p2_avg_hp_when_switching from cache...
Loading p1_max_debuff_received from cache...
Loading p2_max_debuff_received from cache...
Loading p1_avg_move_power from cache...
Loading p2_avg_move_power from cache...
L